<a href="https://colab.research.google.com/github/Shakilkhan24/Playground_DL/blob/main/sqlite_database_with_llm_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sql.py   for resetting db.sqlite



```
# downloading db.sqlite and saving it for later use cases ...
```



In [ ]:
import sqlite3
from langchain.tools import Tool

conn = sqlite3.connect("db.sqlite")    # storage.sqlite



# we can have storage.sqlite from different vectorstores like qdrant,pinecone,faiss etc
# we can download that , and can use for later


def run_sqlite_query(query):
    c = conn.cursor()
    c.execute(query)
    return c.fetchall()


run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="Run a sqlite query.",
    func=run_sqlite_query
)


# using the function ***run_sqlite_query*** as a tool

In [ ]:
# app.py
# interacting db.sqlite for retrieving datas from there


from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from dotenv import load_dotenv

from tools.sql import run_query_tool


load_dotenv()

chat = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

tools = [run_query_tool]

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools
)

agent_executor("How many users are in the database?")
